# Hash the Trash notebook

Here everything should run smoothly with just this notebook, running Ganache (with quickstart) and setting the PORT on which Ganache is running (The contract's abi and bytecode has been compiled from Remix and imported here to run everything)

Various sections are:
* [deploying the contract](#deploy)
* [operating with its functions from eth accounts](#functions)
    * [TARI](#tari)
    * [Trash cycle](#cycle)
    * [Refund](#refund)
* [picking up logs of certain events from the chain](#logs)

In [1]:
# mandatory imports
import json
import random
import pandas as pd
from web3 import Web3
from abi_bytecode import abi, bytecode # saved externally as .py

In [2]:
# Connecting to ganache through opened up PORT
ganache_url = 'HTTP://127.0.0.1:7545'      #change here if different
web3 = Web3(Web3.HTTPProvider(ganache_url))
web3.isConnected()

True

In [3]:
# Compiled abi and bytecode of trash.sol which inherits from citizenz.sol (and Ownable, safemath etc)
abiRemix = json.loads(abi)         # turned to string after copy to clipboard from Remix
bytecodeRemix = bytecode['object'] # it is a dictionary (as copy to clipboard form remix), we use the object for web3 deployment

# simple database example and adding column with ganache accounts
data = pd.read_excel('data/agents_data.xlsx')
data['address'] = web3.eth.accounts     
data

,name,surname,family,mq,weight,recycle,lat,long,role,address
0,Comune,Codogno,0.0,0.0,0.0,NaN,NaN,NaN,municipality,0xe6838FBCc6b637425A493f5d83c3a8ac3c7c176f
1,Francesca,Bianchessi,2.0,70.0,500.0,NaN,NaN,NaN,citizen,0xbc9f917dBb6178524EC0Ae7498589176835B8c92
2,Ilaria,Bolla,4.0,200.0,900.0,NaN,NaN,NaN,citizen,0xb01c26F5ED8E913D1C4399539EC3D89228c640A3
3,Alessandro,Botti,5.0,340.0,800.0,NaN,NaN,NaN,citizen,0x081d3e4F8451824A84A7284133b391d3f2C6b765
4,Davide,Castellini,3.0,50.0,200.0,NaN,NaN,NaN,citizen,0xD9334FE7669D2973844c637c08c4eD8AA9A79366
5,Anna,Di Marco,4.0,80.0,560.0,NaN,NaN,NaN,citizen,0x76416977347C4B799e1710FAb39851489782CC43
6,Truck,Recycle,NaN,NaN,NaN,True,NaN,NaN,truck,0x425C66E726d3350807792FfDF091dF80041CEee0
7,Truck,Not Recycle,NaN,NaN,NaN,False,NaN,NaN,truck,0x2F3183a227c7F609148e7d641D26432aaa1eB173
8,Disposal Station,Milano,NaN,NaN,NaN,True,45.4642,9.1900,station,0x6532fb2598738542829EE93994a69a1Ccc0F8D8f
9,Disposal Station,Bologna,NaN,NaN,NaN,False,44.4949,11.3426,station,0x79Fe1fca9556eb4E222Efaa44A4d4e22f7254Fa2


<a name = 'deploy'></a>

### Deploying the contract

Here we first create a dictionary with the readily available 10 accounts from ganache and then make the municiaplity deploy the contract on the blockchain

In [4]:
def deploy_contract(deployer, _abi, _bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix (aside from getting the compiled abi and bytecode)
    
    Parameters
    -------------
        deployer: eth account
        abi, bytecode : compiled contract things
        
    Returns
    -------------
        contract instance
    """
    contract = web3.eth.contract(abi=_abi, bytecode=_bytecode)      # compiled contract
    tx_hash = contract.constructor().transact({'from': deployer}) # contract constructor call (i.e. deploy)
    
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)      # Get receipt when deployed to blockchain
    print(f"Deployed contract at address:\n{tx_receipt.contractAddress}") # contract address
    
    deployed_ctr = web3.eth.contract(address = tx_receipt.contractAddress, abi  = _abi) # contract
    
    return deployed_ctr

In [5]:
# Deploy contract (municipality in our case) [can check from ganache the blocks to see if it happened]

contract = deploy_contract(deployer = data[data.role == 'municipality']['address'].item(), _abi = abiRemix, _bytecode = bytecodeRemix)

Deployed contract at address:
0x756C2278802DB507AeA4650f94dEd48D1929660A


<a name = 'functions'></a>
## Interacting with the functions

First a simplle check of the owner of the contract and then creating the Agents.

(Still unsure but i think function(...).**call**(...) works only for 'view' functions; need function(...).**transact**(...) for those that modify the chain)

In [6]:
# simple check for owner of the contract
owner = contract.functions.owner().call() # get owner from contract function
print(f"owner: {owner}")
print(f"Is it the municipality? {owner == data[data.role == 'municipality']['address'].item()}")

# to remember inputs and all functions
contract.all_functions()

owner: 0xe6838FBCc6b637425A493f5d83c3a8ac3c7c176f
Is it the municipality? True


[<Function MunicipalityBalance()>,
 <Function TariAmount(address)>,
 <Function citizens(address)>,
 <Function createCitizen(address,string,uint256,uint256,uint256)>,
 <Function createStation(address,bool,int256,int256)>,
 <Function createTruck(address,bool)>,
 <Function deleteCitizen(address)>,
 <Function deleteStation(address)>,
 <Function deleteTruck(address)>,
 <Function drop(address,int256,int256)>,
 <Function givePayout(address)>,
 <Function numberC()>,
 <Function numberS()>,
 <Function numberT()>,
 <Function owner()>,
 <Function payTari()>,
 <Function pick(address,uint256,uint256)>,
 <Function received(bool,address,uint256)>,
 <Function renounceOwnership()>,
 <Function stations(address)>,
 <Function transferOwnership(address)>,
 <Function trucks(address)>,
 <Function withdraw()>]

In [7]:
# Creating list of touples as correct inputs for the contract structs

# Create CITIZENS - (address payable _address, string memory _name, uint _family, uint _house, uint256 _w)
citizensL = [[r.address, " ".join([r.name, r.surname]), int(r.family), int(r.mq), int(r.weight)] 
             for r in data.itertuples() if r.role == 'citizen']

# Create TRUCKS - (address _address, bool _recycle)
trucksL = [[r.address, r.recycle] for r in data.itertuples() if r.role == 'truck']

# Create STATIONS - (address _address, bool _recycle, int _lat, int _long)
stationsL = [[r.address, r.recycle, int(r.lat), int(r.long)] for r in data.itertuples() if r.role == 'station']

print(f'Example of createCitizen inputs: {citizensL[0]}')

Example of createCitizen inputs: ['0xbc9f917dBb6178524EC0Ae7498589176835B8c92', 'Francesca Bianchessi', 2, 70, 500]


In [8]:
def create(municip_addr ,c, t, s):
    '''
    Calls contract funcions to assigns addresses (and their characteristics) roles and populate mappings
    
    Parameters
    ----------------------
        municip_addr : eth address of the municipality
        c, t, s : list of touples with correct characteristics
        
    Returns
    ----------------------
        nothing on python, the chain grows as these transactions are registered
    '''
    from_dict = {'from': municip_addr}
    for i in range(len(c)):
        contract.functions.createCitizen(c[i][0], c[i][1], c[i][2], c[i][3], c[i][4]).transact(from_dict)
    for j in range(len(t)):
        contract.functions.createTruck(t[j][0], t[j][1]).transact(from_dict)
    for k in range(len(s)):
        contract.functions.createStation(s[k][0], s[k][1], s[k][2], s[k][3]).transact(from_dict)
    print('Done')

In [9]:
# Populating mappings and check amount of entities
create(data[data.role == 'municipality']['address'].item(),citizensL, trucksL, stationsL)

print(f"""
# Citizens: {contract.functions.numberC().call()} 
# Trucks : {contract.functions.numberT().call()}
# PLants : {contract.functions.numberS().call()}""")

Done

# Citizens: 5 
# Trucks : 2
# PLants : 2


In [10]:
# Some functions to show structs statuses as we change them
def get_citizen_info(address):
    """
    Call to the contract and retrive (pretty) informations on citizen
    """
    info = contract.functions.citizens(address).call()
    print(f"""
    Address eth : {address}
    Name : {info[0]}
    Family members : {info[1]}
    House sq.meters : {info[2]}
    Assigned weight\liters : {info[3]}
    TARI amount : {info[4]}
    Recyclable Waste Tot : {info[5]}
    Non Recyclable Waste Tot : {info[6]}
    Paid TARI : {info[7]}
    Active account : {info[8]}
    """)

def get_truck_info(address):
    """Show pretty info on trucks"""
    info = contract.functions.trucks(address).call()
    print(f"""
    Address eth : {address}
    Truck number : {info[0]}
    Weight transported : {info[1]}
    Recyclable Truck : {info[2]}
    Active Truck : {info[3]}
    """)
    
    
def get_station_info(address):
    """Show pretty info for stations"""
    info = contract.functions.stations(address).call()
    print(f"""
    Address eth : {address}
    Station nr. : {info[0]}
    Weight : {info[1]}
    latitude : {info[2]}
    longitude : {info[3]}
    Recyclable Plant : {info[4]}
    Active Plant : {info[5]}
    """)

In [11]:
# using exemplar citizen to do check along the road
dutiful_citizen = data[data.role == 'citizen']['address'].reset_index(drop = True)[0]
get_citizen_info(dutiful_citizen)

ex_truck = data[data.role == 'truck']['address'].reset_index(drop = True)[0]
get_truck_info(ex_truck)

ex_station = data[data.role == 'station']['address'].reset_index(drop = True)[0]
get_station_info(ex_station)


    Address eth : 0xbc9f917dBb6178524EC0Ae7498589176835B8c92
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 0
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    

    Address eth : 0x425C66E726d3350807792FfDF091dF80041CEee0
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    

    Address eth : 0x6532fb2598738542829EE93994a69a1Ccc0F8D8f
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


<a name = 'tari'></a>
### Compute TARI 

In [12]:
# Calculate amount of TARI for each citizen (and looping for the others)
municipality = data[data.role == 'municipality']['address'].item()

for addr in data[data.role == 'citizen']['address']:
    contract.functions.TariAmount(addr).transact({'from' : municipality})

In [13]:
get_citizen_info(data[data.role == 'citizen']['address'].reset_index(drop = True)[0])
"""looks expensive"""


    Address eth : 0xbc9f917dBb6178524EC0Ae7498589176835B8c92
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    


'looks expensive'

In [14]:
# Each citizen pays its due amount
for addr in data[data.role == 'citizen']['address']:
    
    info = contract.functions.citizens(addr).call() # need the amount from the contract
    print(f"Citizen {info[0]} is paying {info[4]} wei")
    
    # payable function, the amount is passed in the dictionary of .transact()
    contract.functions.payTari().transact({'from': addr, 'value' : info[4]})
    

Citizen Francesca Bianchessi is paying 190000000000000000 wei
Citizen Ilaria Bolla is paying 490000000000000000 wei
Citizen Alessandro Botti is paying 1440000000000000000 wei
Citizen Davide Castellini is paying 120000000000000000 wei
Citizen Anna Di Marco is paying 216000000000000000 wei


In [15]:
get_citizen_info(data[data.role == 'citizen']['address'].reset_index(drop = True)[0])
"FINALLY PAID"


    Address eth : 0xbc9f917dBb6178524EC0Ae7498589176835B8c92
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : True
    Active account : True
    


'FINALLY PAID'

<a name = 'cycle'></a>
## Working with trashbags lifecycles

The trucks will pick them up and drop them at the disposal station. From excel files we simulate what the sensors would use as inputs.

In [16]:
bags = pd.read_excel("data/bags_data.xlsx")
bags.head(10) 

,name,surname,weight,recycle
0,Francesca,Bianchessi,1,True
1,Ilaria,Bolla,2,False
2,Alessandro,Botti,3,True
3,Davide,Castellini,1,False
4,Anna,Di Marco,2,True
5,Francesca,Bianchessi,1,True
6,Ilaria,Bolla,6,False
7,Alessandro,Botti,4,True
8,Davide,Castellini,3,False
9,Anna,Di Marco,2,True


#### Pick up trash

Function called from the truck and using sensors to check the weight 

Here we automatically assign the correct truck to the trash type. 
##### ! if the two don't match we have a slight problem !

In [17]:
# The right truck will pick up the various trashbags
for i, name, sur, w, recyclable in bags.itertuples():

    # get address of generator
    generator_addr = data[data.name == name]['address'].item()
    
    # get correct truck address via subsetting
    truck_addr = data[(data['name'] == 'Truck') & (data['recycle'] == recyclable)]['address'].item()
    
    # 'pick' function in contract is called by the correct truck
    contract.functions.pick(generator_addr, w, random.randint(0, 1e+10)).transact({'from' : truck_addr})

In [18]:
get_citizen_info(dutiful_citizen)
get_truck_info(ex_truck)

"""See both counters increased"""


    Address eth : 0xbc9f917dBb6178524EC0Ae7498589176835B8c92
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 16
    Non Recyclable Waste Tot : 12
    Paid TARI : True
    Active account : True
    

    Address eth : 0x425C66E726d3350807792FfDF091dF80041CEee0
    Truck number : 1
    Weight transported : 71
    Recyclable Truck : True
    Active Truck : True
    


'See both counters increased'

#### Drop bags at station

Still a function from the side of the truck. The truck will result empty after dropping and the station will increase its total counter with the recieved weight.

In [19]:
gps = pd.read_excel("data/gps_data.xlsx")
gps

,name,surname,lat,long
0,Disposal Station,Milano,45.4642,9.1900
1,Disposal Station,Bologna,44.4949,11.3426


In [20]:
get_station_info(ex_station)


    Address eth : 0x6532fb2598738542829EE93994a69a1Ccc0F8D8f
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


In [21]:
# get station name and coords from the gps
for i, name, sur, lat, long in gps.itertuples():
    
    #get station address
    station_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    
    #get_station_info(station_addr)
    # need to get the type of station (recycling/not recycling) to pair the truck
    s_info = contract.functions.stations(station_addr).call() # s_info[4] is the type
    
    # pairing truck based on recyclable or not and calling the 'drop' function
    correct_truck = data[(data.name == 'Truck') & (data.recycle == s_info[4])]['address'].item()
    
    contract.functions.drop(station_addr, int(lat), int(long)).transact({'from' : correct_truck})
    

In [22]:
get_station_info(ex_station)
get_truck_info(ex_truck)


    Address eth : 0x6532fb2598738542829EE93994a69a1Ccc0F8D8f
    Station nr. : 1
    Weight : 71
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    

    Address eth : 0x425C66E726d3350807792FfDF091dF80041CEee0
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    


#### Recieved 

Function called by the station. Its sensors will say how much they actually recieved and from which truck.(This excel is from the point of view of a single station, just to mess it up on purpose.)

##### ! If the weight detected is not the same as it was in the truck it reverts. This is a problem for the next truck that stops there; since it checks the current total weight at the station which is always higher after the first truck !

In [23]:
station_sense = pd.read_excel("data/sensors_data.xlsx")
station_sense

,name,surname,weight
0,Truck,Recycle,71
1,Truck,Not Recycle,61


In [24]:
for i, name, sur ,weight in station_sense.itertuples():
    
    # get truck address
    truck_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    info_tr = contract.functions.trucks(truck_addr).call() # to get truck type in info_tr[2]
    
    
    # station address is the same both times [can change True with info_tr[2] to get the correct station to read the correct truck]
    st_addr = data[(data.name == 'Disposal Station') & (data.recycle == True)]['address'].item() 
    print(f" Truck: {truck_addr}\nStation: {st_addr}")
    
    #recieved function call
    contract.functions.received(info_tr[2], truck_addr, weight).transact({'from' : st_addr})
    
    

 Truck: 0x425C66E726d3350807792FfDF091dF80041CEee0
Station: 0x6532fb2598738542829EE93994a69a1Ccc0F8D8f
 Truck: 0x2F3183a227c7F609148e7d641D26432aaa1eB173
Station: 0x6532fb2598738542829EE93994a69a1Ccc0F8D8f


SolidityError: execution reverted: VM Exception while processing transaction: revert

<a name = 'refund'></a>
## Refund

municipality withdraws form the contract the ETH and then does the payout for all citizens


In [25]:
print(municipality) # we have the municipality address stored from before

# the withdraw works just like this, see ganache '0' account --> municipality
before = web3.eth.getBalance(municipality)
contract.functions.withdraw().transact({'from' : municipality}) 
after = web3.eth.getBalance(municipality)
print(f"Withdrawn: {after-before} (also minus gas costs)")

0xe6838FBCc6b637425A493f5d83c3a8ac3c7c176f
Withdrawn: 2455404660000000000 (also minus gas costs)


In [26]:
# citizen address as input to function
for citizen in data[data.role == 'citizen']['address']:
    
    # balance comparison for the refund
    before = web3.eth.getBalance(citizen)
    contract.functions.givePayout(citizen).transact({'from' : municipality})
    after = web3.eth.getBalance(citizen)
    
    print(f"{citizen}\nBefore: {before}\nAfter : {after}\nRefunded:{after-before}")
        

0xbc9f917dBb6178524EC0Ae7498589176835B8c92
Before: 99809569060000000000
After : 99809569060000000000
Refunded:0
0xb01c26F5ED8E913D1C4399539EC3D89228c640A3
Before: 99509569060000000000
After : 99509569060000000000
Refunded:0
0x081d3e4F8451824A84A7284133b391d3f2C6b765
Before: 98559569060000000000
After : 98559569060000000000
Refunded:0
0xD9334FE7669D2973844c637c08c4eD8AA9A79366
Before: 99879569060000000000
After : 99879569060000000000
Refunded:0
0x76416977347C4B799e1710FAb39851489782CC43
Before: 99783569060000000000
After : 99783569060000000000
Refunded:0


In [ ]:
## Right now givePayout isn't really working

<a name = 'logs'></a>

## Event/Log Filtering

After running the above cells the chain is populated with a lot of different logs from events. They can be extracted with specific filters that are contract specific. 

(Right now just 1 filter for the sake of demonstration. A ***real time*** version that repeatedly checks the last block still needs working)

In [27]:
# creating logs/event filters to inspect the whole chain (fromBlock = 1)

# filter for the Event 'PickedUp' generated by the truck
pickedUp_filter = contract.events.PickedUp.createFilter(fromBlock = 1)
#recieved_filter etc

filters = [pickedUp_filter]

In [28]:
def get_past_logs(filter_list):
    """
    Iterates over every filter built and extracts logs from the block specified in the filter to the 'latest'
    
    inputs
    ---------------
     filter_list : filters created
    
    returns
    ---------------
     list containing every event attribute generated from the 'emit' on the contract
    """
    events = []
    for event_filter in filter_list:
        for e in event_filter.get_all_entries(): # get_new_entry() to check only last block
            
            # e is a nested dictionary, like this we bring all elements on the same level
            args = dict(e['args'])
            args.update(dict(e))
            del args['args'] # brought one level above, delete the nest
            # args.pop('args', None) # could delete like this too
            
            events.append(args)
            
    return events

In [29]:
# Pass filters on the chain (from block 1) and pick logs found (for now only 'ToPickUp')
# Can see logs of the 5 citizens, each one a different generator and all the garble generated by the event
events = get_past_logs(filters)
df = pd.DataFrame(events)
df

,transporter,wasteType,bagId,generator,wasteWeight,pickUpTime,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber
0,0x425C66E726d3350807792FfDF091dF80041CEee0,True,"b""\xb1;[V-\x08\x15'N\xa0\x7f\xa0\x1a\x9e\x98.\...",0xbc9f917dBb6178524EC0Ae7498589176835B8c92,1,1607554138,PickedUp,0,0,"b'\x88\xed\xa8N`\x05\x1c\x073\xb6""\xc8\xc2?\x1...",0x756C2278802DB507AeA4650f94dEd48D1929660A,"b""\xb0\x1a\xa0\xd4\xefG\x12\x9e\xdb\xf6\xb3\xe...",21
1,0x2F3183a227c7F609148e7d641D26432aaa1eB173,False,b'o6\x14\xfd\xc1\x88\x11\xef\x84dH\x13\xaee\xc...,0xb01c26F5ED8E913D1C4399539EC3D89228c640A3,2,1607554138,PickedUp,0,0,b'\x85\xa2\x1aV5\x0e<&\xa3\x801\x7f\xe6\x04\x9...,0x756C2278802DB507AeA4650f94dEd48D1929660A,b'7\x15\xc0;>\xf8\xf3+\xcc\xafn\xdcV\xdb\xc4\x...,22
2,0x425C66E726d3350807792FfDF091dF80041CEee0,True,b'n\x0e\x89\xfbm\xb6\xe9I\x1f*\xa7\xf0\xea\xeb...,0x081d3e4F8451824A84A7284133b391d3f2C6b765,3,1607554139,PickedUp,0,0,b'\xeeg-y\xea\xca3\xb9&\xe0\xa4\xb1D\xfb\xa1\x...,0x756C2278802DB507AeA4650f94dEd48D1929660A,"b""\xb9M6\xd2\xd8{>\xea^x^;\xebvT\x84\x85\xb5\x...",23
3,0x2F3183a227c7F609148e7d641D26432aaa1eB173,False,b'>F\xcd\x89\x8f\xbb3B\xc9\x8e\xfem\x91\x95>\x...,0xD9334FE7669D2973844c637c08c4eD8AA9A79366,1,1607554139,PickedUp,0,0,b'\x13\xf1\x85\xcb\x98\x1e\x8b\xcfx\xa2lJ_\xec...,0x756C2278802DB507AeA4650f94dEd48D1929660A,b'\xca\xe7\x9d.lg\xd3\xa5\xa7\xd9.\xa1\xa8\x15...,24
4,0x425C66E726d3350807792FfDF091dF80041CEee0,True,b'\xf5\xe4\xdcC9\xb6=\xda%\xc9\xac<m\xb5\x8f\x...,0x76416977347C4B799e1710FAb39851489782CC43,2,1607554139,PickedUp,0,0,b'W\xd4M\xf7\x01#e\xed\x9e\xb7\xaa\x8b\xcf\xe4...,0x756C2278802DB507AeA4650f94dEd48D1929660A,b'\xe4\xbc\x92\x16\x05G\x82\xa9\xdc5d\xd8`\x8e...,25
5,0x425C66E726d3350807792FfDF091dF80041CEee0,True,b'`\x84\xa3\x9e\xaa!\xbdv\x86\xdc\xd2\x88\xd0\...,0xbc9f917dBb6178524EC0Ae7498589176835B8c92,1,1607554139,PickedUp,0,0,b'#\xa8\xfbj\xe8\xb5|>\xb0\xf32\xbejsR\xfe>m\x...,0x756C2278802DB507AeA4650f94dEd48D1929660A,"b""E\xd9\xe3n\xcb\xea)\x9c\xa7\x83\xf3\xc1\x18\...",26
6,0x2F3183a227c7F609148e7d641D26432aaa1eB173,False,b'\xe5\xd2\x1a\xa6\xcf\xe6\xdfx\xd1\xde\xa4_\x...,0xb01c26F5ED8E913D1C4399539EC3D89228c640A3,6,1607554139,PickedUp,0,0,b'\x92:\xec\x0fo\xff\xfa\xb4v1}\x01\xbcG|aq\xe...,0x756C2278802DB507AeA4650f94dEd48D1929660A,"b'\n\xca\x97\xe1\xfd\x0c\xfe\xc2,$g5\x88\xfb#\...",27
7,0x425C66E726d3350807792FfDF091dF80041CEee0,True,"b""\x8f\xc4 I\x11\xb8\xb5\xad'\xc6\x85\x8b\x983...",0x081d3e4F8451824A84A7284133b391d3f2C6b765,4,1607554140,PickedUp,0,0,b'l\xfc4#\xc4\xb9*\xe6\xdb\xb0>v\xad\x19\x86S&...,0x756C2278802DB507AeA4650f94dEd48D1929660A,b'y\xec\xf24\x12D\xfd\xc3Z\xfbZ\xe9\x1aC?\xba\...,28
8,0x2F3183a227c7F609148e7d641D26432aaa1eB173,False,b'\xd2\xd6\xb4\xf5\xca\xc7Yh(\xf1\x08\xfc\x08$...,0xD9334FE7669D2973844c637c08c4eD8AA9A79366,3,1607554140,PickedUp,0,0,b'\xd69q\xf1\xf3Ex\x7f\x9a\x088_\xb3\xd1+\x9fN...,0x756C2278802DB507AeA4650f94dEd48D1929660A,"b'\x96T@\xfc\x1e\x86,\xb0\x1a\xdf\xc9\xba\x0eC...",29
9,0x425C66E726d3350807792FfDF091dF80041CEee0,True,b'vxB\xfc\xe7\x16\x1b1\n\xd4V\x00_^\x12\x08\x8...,0x76416977347C4B799e1710FAb39851489782CC43,2,1607554140,PickedUp,0,0,"b'H,\x0c\x04\xe7\xb0L\x87\xc9eh\xe7 \xc0\xcdm\...",0x756C2278802DB507AeA4650f94dEd48D1929660A,b'\x03}iM\xd8\xe9\x05\xaf/I;\xda\xd8\x0c\x1cc\...,30
